# Blockchain RAG

In [2]:
!pip install ragatouille
!pip install chromadb
!pip install langchain langchain-community langchain-huggingface --upgrade
!pip install web3


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [47]:
import chromadb
import pandas as pd
from ragatouille import RAGPretrainedModel
import ollama
from typing import Optional
from web3 import Web3
import requests
import hashlib
from langchain.text_splitter import TokenTextSplitter

## Retrieve the Corpus from the Blockchain

### On-Chain

In [2]:
abi = [
	{
		"inputs": [],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "corpus",
		"outputs": [
			{
				"internalType": "address",
				"name": "sender",
				"type": "address"
			},
			{
				"internalType": "string",
				"name": "metadata",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256[]",
				"name": "ids",
				"type": "uint256[]"
			},
			{
				"internalType": "string",
				"name": "metadata",
				"type": "string"
			},
			{
				"internalType": "string[]",
				"name": "chunks",
				"type": "string[]"
			}
		],
		"name": "insert",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "key",
				"type": "uint256"
			}
		],
		"name": "removeDocument",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "sender",
				"type": "address"
			}
		],
		"name": "removeDocumentBySender",
		"outputs": [
			{
				"internalType": "uint256[]",
				"name": "",
				"type": "uint256[]"
			}
		],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "key",
				"type": "uint256"
			}
		],
		"name": "retrieveDocument",
		"outputs": [
			{
				"components": [
					{
						"internalType": "address",
						"name": "sender",
						"type": "address"
					},
					{
						"internalType": "string",
						"name": "metadata",
						"type": "string"
					},
					{
						"internalType": "uint256[]",
						"name": "ids",
						"type": "uint256[]"
					},
					{
						"internalType": "string[]",
						"name": "chunks",
						"type": "string[]"
					}
				],
				"internalType": "struct Corpus.Document",
				"name": "",
				"type": "tuple"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "sender",
				"type": "address"
			}
		],
		"name": "retrieveDocumentKeysBySender",
		"outputs": [
			{
				"internalType": "uint256[]",
				"name": "",
				"type": "uint256[]"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "retrieveLatestID",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "retrieveLatestKey",
		"outputs": [
			{
				"internalType": "int256",
				"name": "",
				"type": "int256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]

In [3]:
web3class = Web3(Web3.HTTPProvider('http://localhost:8545'))
contractAddress = web3class.to_checksum_address('0x5fbdb2315678afecb367f032d93f642f64180aa3')

In [14]:
def retrieveDocument(key):
    contract = web3class.eth.contract(abi=abi, address=contractAddress)
    key = key
    doc = contract.functions.retrieveDocument(key).call()
    if (len(doc[2]) > 0):
        result = f'Retrieved document {key} results'
        print(result)
        return doc[1], doc[2], doc[3]
    else:
        result = f'No results found for {key}'
        print(result)
        return None, None, None

In [5]:
metadata, ids, docs = retrieveDocument(1)

Retrieved document 1 results
Metadata: ghibli example 1
IDs: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
Chunks: ["Hayao Miyazaki was born on January 5, 1941, in the town Akebono-cho in Hongō, Tokyo City, Empire of Japan, the second of four sons. His father, Katsuji Miyazaki (born 1915), was the director of Miyazaki Airplane, his brother's company, which manufactured rudders for fighter planes during World War II. The business allowed his family to remain affluent during Miyazaki's early life. Miyazaki's father enjoyed purchasing paintings and demonstrating them to guests, but otherwise had little known artistic understanding. He was in the Imperial Japanese Army around 1940, discharged and lectured about disloyalty after declaring to his commanding officer that he wished not to fight because of his wife and young child. According to Miyazaki, his father often told him about his exploits, claiming he conti

In [42]:
def retrieveCorpus():
    contract = web3class.eth.contract(abi=abi, address=contractAddress)
    latest_key = contract.functions.retrieveLatestKey().call();
    docs = []
    ids = []
    metadatas = []
    for key in range(latest_key+1):
        doc_metadata, doc_ids, doc_chunks = retrieveDocument(key)
        if doc_chunks is not None:
            for doc in range(len(doc_chunks)):
                docs.append(doc_chunks[doc])
                ids.append(str(doc_ids[doc]))
                metadatas.append({'doc_type':doc_metadata})
    return metadatas, ids, docs

In [43]:
db_metadatas, db_ids, db_docs = retrieveCorpus()

Retrieved document 0 results
Retrieved document 1 results


In [44]:
print(db_metadatas)
print(db_ids)
print(db_docs)

[{'doc_type': 'init'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli exam

### Off-Chain

In [1]:
abi = [
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "corpus",
		"outputs": [
			{
				"internalType": "address",
				"name": "sender",
				"type": "address"
			},
			{
				"internalType": "string",
				"name": "metadata",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "hash",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "location",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "string",
				"name": "metadata",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "hash",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "location",
				"type": "string"
			}
		],
		"name": "insert",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "key",
				"type": "uint256"
			}
		],
		"name": "removeDocument",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "sender",
				"type": "address"
			}
		],
		"name": "removeDocumentBySender",
		"outputs": [
			{
				"internalType": "uint256[]",
				"name": "",
				"type": "uint256[]"
			}
		],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "key",
				"type": "uint256"
			}
		],
		"name": "retrieveDocument",
		"outputs": [
			{
				"components": [
					{
						"internalType": "address",
						"name": "sender",
						"type": "address"
					},
					{
						"internalType": "string",
						"name": "metadata",
						"type": "string"
					},
					{
						"internalType": "string",
						"name": "hash",
						"type": "string"
					},
					{
						"internalType": "string",
						"name": "location",
						"type": "string"
					}
				],
				"internalType": "struct HashCorpus.Document",
				"name": "",
				"type": "tuple"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "sender",
				"type": "address"
			}
		],
		"name": "retrieveDocumentKeysBySender",
		"outputs": [
			{
				"internalType": "uint256[]",
				"name": "",
				"type": "uint256[]"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "retrieveLatestKey",
		"outputs": [
			{
				"internalType": "int256",
				"name": "",
				"type": "int256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]

In [4]:
web3class = Web3(Web3.HTTPProvider('http://localhost:8545'))
contractAddress = web3class.to_checksum_address('0x5fbdb2315678afecb367f032d93f642f64180aa3')

In [5]:
def retrieveDocument(key):
    contract = web3class.eth.contract(abi=abi, address=contractAddress)
    key = key
    doc = contract.functions.retrieveDocument(key).call()
    if (len(doc[0]) > 0):
        result = f'Retrieved document {key} results'
        print(result)
        return doc[1], doc[2], doc[3]
    else:
        result = f'No results found for {key}'
        print(result)
        return None, None, None

In [35]:
metadata, doc_hash, location = retrieveDocument(7)
print(metadata)
print(doc_hash)
print(location)

Retrieved document 7 results
example
edb0f70ad878a3d57d4973dfd379d175604f4591bfacae37c2d8470274256794
Studio_Ghibli.txt


In [20]:
def downloadDocument(location):
    params = {'filename':f'{location}'}
    url = 'http://localhost:8080/download/'
    return requests.get(url, params).text

In [36]:
doc = downloadDocument('Studio_Ghibli.txt')
print(doc)

Studio Ghibli, Inc. (Japanese: 株式会社スタジオジブリ, Hepburn: Kabushiki-gaisha Sutajio Jiburi) is a Japanese animation studio based in Koganei, Tokyo. It has a strong presence in the animation industry and has expanded its portfolio to include various media formats, such as short subjects, television commercials, and two television films. Their work has been well-received by audiences and recognized with numerous awards. Their mascot and most recognizable symbol, the character Totoro from the 1988 film My Neighbor Totoro, is a giant spirit inspired by raccoon dogs (tanuki) and cats (neko). Among the studio's highest-grossing films are Princess Mononoke (1997), Spirited Away (2001), Howl's Moving Castle (2004), Ponyo (2008), and The Boy and the Heron (2023). Studio Ghibli was founded on June 15, 1985, by the directors Hayao Miyazaki and Isao Takahata and producer Toshio Suzuki, after acquiring Topcraft's assets.
Four of the studio's films are among the ten highest-grossing Japanese feature films

Original JavaScript hash for 0: b4723cf013fcbdc5a7901d2595d6df1cb6ae387e7b6cb29235bf3dd88c1712a8

Original JavaScript hash for 7: edb0f70ad878a3d57d4973dfd379d175604f4591bfacae37c2d8470274256794

In [37]:
sha256 = hashlib.sha256()
string = str.encode(doc)
sha256.update(string)
string_hash = sha256.hexdigest()
print(string_hash)

edb0f70ad878a3d57d4973dfd379d175604f4591bfacae37c2d8470274256794


In [57]:
def retrieveHashCorpus():
    contract = web3class.eth.contract(abi=abi, address=contractAddress)
    latest_key = contract.functions.retrieveLatestKey().call();
    docs = []
    metadatas = []
    for key in range(latest_key+1):
        doc_metadata, doc_hash, doc_location = retrieveDocument(key)
        if doc_hash is not None:
            doc = downloadDocument(doc_location)
            sha256 = hashlib.sha256()
            hash_string = str.encode(doc)
            sha256.update(hash_string)
            calc_hash = sha256.hexdigest()
            if doc_hash == calc_hash:
                docs.append(doc)
                metadatas.append(doc_metadata)
                print(f'Valid hash found for document {key}, document retrieved')
            else:
                print(f'Invalid hash found for document {key}, document not retrieved')
                print(f'Blockchain hash: {doc_hash}')
                print(f'Calculated hash: {calc_hash}')
    return metadatas, docs

In [58]:
raw_metadatas, raw_docs = retrieveHashCorpus()

Retrieved document 0 results
Valid hash found for document 0, document retrieved
Retrieved document 1 results
Valid hash found for document 1, document retrieved
Retrieved document 2 results
Valid hash found for document 2, document retrieved
Retrieved document 3 results
Valid hash found for document 3, document retrieved
Retrieved document 4 results
Valid hash found for document 4, document retrieved
Retrieved document 5 results
Valid hash found for document 5, document retrieved
Retrieved document 6 results
Valid hash found for document 6, document retrieved
Retrieved document 7 results
Valid hash found for document 7, document retrieved
Retrieved document 8 results
Valid hash found for document 8, document retrieved


In [59]:
print(raw_docs)

['Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, Japanese: [mijaꜜzaki hajao]; born January 5, 1941) is a Japanese animator, filmmaker, and manga artist. A founder of Studio Ghibli, he has attained international acclaim as a masterful storyteller and creator of Japanese animated feature films, and is widely regarded as one of the most accomplished filmmakers in the history of animation.\nBorn in Tokyo City, Miyazaki expressed interest in manga and animation from an early age. He joined Toei Animation in 1963, working as an inbetween artist and key animator on films like Gulliver\'s Travels Beyond the Moon (1965), Puss in Boots (1969), and Animal Treasure Island (1971), before moving to A-Pro in 1971, where he co-directed Lupin the Third Part I (1971–1972) alongside Isao Takahata. After moving to Zuiyō Eizō (later Nippon Animation) in 1973, Miyazaki worked as an animator on World Masterpiece Theater and directed the television series Future Boy Conan (1978). He joined Tokyo Movie Shinsha 

In [66]:
chunker = TokenTextSplitter(chunk_size=1000, chunk_overlap=100)
db_docs = []
db_metadatas = []
db_ids = []
ids = 0
for doc in range(len(raw_docs)):
    split = chunker.split_text(raw_docs[doc])
    for chunked_doc in range(len(split)):
        db_docs.append(split[chunked_doc])
        db_metadatas.append({'document_metadata': raw_metadatas[doc]})
        db_ids.append(f'{ids}')
        ids += 1

In [64]:
print(len(db_docs))
print(len(db_metadatas))
print(len(db_ids))

125
125
125


In [56]:
print(chunked_docs[:5])

['Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, Japanese: [mijaꜜzaki hajao]; born January 5, 1941) is a Japanese animator, filmmaker, and manga artist. A founder of Studio Ghibli, he has attained international acclaim as a masterful storyteller and creator of Japanese animated feature films, and is widely regarded as one of the most accomplished filmmakers in the history of animation.\nBorn in Tokyo City, Miyazaki expressed interest in manga and animation from an early age. He joined Toei Animation in 1963, working as an inbetween artist and key animator on films like Gulliver\'s Travels Beyond the Moon (1965), Puss in Boots (1969), and Animal Treasure Island (1971), before moving to A-Pro in 1971, where he co-directed Lupin the Third Part I (1971–1972) alongside Isao Takahata. After moving to Zuiyō Eizō (later Nippon Animation) in 1973, Miyazaki worked as an animator on World Masterpiece Theater and directed the television series Future Boy Conan (1978). He joined Tokyo Movie Shinsha 

## Preprocess and Load the Corpus into the Vector Store

In [45]:
chroma_client = chromadb.Client()
#chroma_client.delete_collection(name="ghibli") # In case we need to redo anything
collection = chroma_client.get_or_create_collection(name="ghibli")
collection.add(documents=db_docs, ids=db_ids, metadatas=db_metadatas)

In [46]:
# Display an example search against chromadb

results = collection.query(
    query_texts=["What was Miyazaki's first film?"],
    n_results=2
)
print(results)

{'ids': [['16', '17']], 'distances': [[0.770384669303894, 0.7783008217811584]], 'metadatas': [[{'doc_type': 'ghibli example 1'}, {'doc_type': 'ghibli example 1'}]], 'embeddings': None, 'documents': [["In 1987, Studio Ghibli acquired the rights to create a film adaptation of Eiko Kadono's novel Kiki's Delivery Service. Miyazaki's work on My Neighbor Totoro prevented him from directing the adaptation; he acted as producer, while Sunao Katabuchi was chosen as director and Nobuyuki Isshiki as script writer. Miyazaki's dissatisfaction of Isshiki's first draft led him to make changes to the project, ultimately taking the role of director. Kadono expressed her dissatisfaction with the differences between the book and screenplay, but Miyazaki and Takahata convinced her to let production continue. The film was originally intended to be a 60-minute special, but expanded into a feature film after Miyazaki completed the storyboards and screenplay. Miyazaki felt the struggles of the protagonist, Ki

## Define the RAG

In [47]:
# Define the reranker

llmreranker = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

/home/mist861/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/mist861/anaconda3/lib/python3.11/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


In [51]:
# The below takes the prompt, performs a retrieval against chromadb, reranks it, then calls ollama. I did not make (most) of this, it's derived from: https://huggingface.co/learn/cookbook/en/advanced_rag

def rag_query(
    question: str,
    llm: str,
    knowledge_index=collection,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 10,
    num_docs_final: int = 5):
    print("=> Retrieving documents...")
    relevant_docs = knowledge_index.query(query_texts=question, n_results=num_retrieved_docs)
    relevant_docs = relevant_docs['documents'][0]

    if reranker:
        print("=> Reranking documents...")
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)

    relevant_docs = relevant_docs[:num_docs_final]

    final_prompt = f"""
        use the following CONTEXT to answer the QUESTION at the end.
        If you don't know the answer, just say that you don't know, don't try to make up an answer.
        
        CONTEXT: {relevant_docs}
        QUESTION: {question}
        
        """

    response = ollama.chat(model=llm, messages=[
        {
            'role': 'user',
            'content': final_prompt,
        },
    ])
    answer = response['message']['content']

    return answer, relevant_docs

In [49]:
# Perform a retrieval/generations without the reranker

question="What was Miyazaki's first film?"
result, docs = rag_query(question, 'llama3.1')
print(result)

=> Retrieving documents...


In [50]:
print(result)

I don't know. The context doesn't mention his first film explicitly, only that "Porco Rosso" was not his first film (as it did not top Animage's yearly reader poll).


In [ ]:
# Display the retrieved documents used

print(docs)
len(docs)

In [52]:
# Perform a retrieval/generation with the reranker

result, docs = rag_query(question, 'llama3.1', reranker=llmreranker)
print(result)

=> Retrieving documents...
=> Reranking documents...


/home/mist861/anaconda3/lib/python3.11/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 27.36it/s]


Based on the context, Miyazaki's first feature anime film was "The Castle of Cagliostro" (1979), which he directed for Tokyo Movie Shinsha.
